# Solving stochastic resources-constrainded project scheduling problems with a Neural Network based on Reinforcement Learning methods

In [2]:
import pandas as pd
import re # regex
from random import random, shuffle, choice
from itertools import chain, combinations, compress
import tensorflow as tf

## Building the input DataFrames
* a DataFrame *general_df* for general project data (link to base data and inital value generator)
* a DataFrame *project_df* with project data, like number of jobs and the due date
* a DataFrame *resources_df* which is a list of all resources and their limits
* a DataFrame *df* that contains all the network info: all nodes with their successors and predecessors, their duration, etc.
* a DataFrame *limits* that is used to keep track of available resources and resources in use

In [4]:
#path = r'D:\Gregor\Dropbox\Studium\j120.sm\j12060_9.sm'
path = r'C:\Users\webgr_000\Dropbox\Studium\[M04] - WS 2018-2019\Thesis\Test Sets\J30\J30.sm\j301_1.sm'
#path = r'D:\Gregor\Dropbox\Studium\[M04] - WS 2018-2019\Thesis\Test Sets\J30\J30.sm\j301_1.sm'

with open(path, "r") as f:
    file = f.readlines()

In [5]:
general = {}
for line in file[1:3]:
    l = list(map(str.strip, line.split(sep = ':')))
    general[l[0]] = l[1]

project = {}
for line in file[4:7]:
    l = list(map(str.strip, line.split(sep = ':')))
    project[l[0]] = l[1]

info = (list(map(str.split, file[13:15])))
for i in range(len(info[0])):
    project[info[0][i]] = info[1][i]

general_df = pd.DataFrame.from_dict(general, orient='index', columns=['value'])
project_df = pd.DataFrame.from_dict(project, orient='index', columns=['value'])

In [6]:
res_sep = re.compile('[RND][ \d]{2}|[\d]+') # regex pattern to extract the resources
resources = {}
for line in file[8:11]:
    l = list(map(str.strip, line.replace('-', '').split(sep = ':')))
    resources[l[0]] = l[1][0]
res = [re.findall(res_sep, file[-3]), re.findall(res_sep, file[-2])]
for i, r in enumerate(res[0]):
    resources[r] = res[1][i]
resources_df = pd.DataFrame.from_dict(resources, orient='index', columns=['value'])

In [7]:
sep = re.compile('[*]+') # regex pattern to find the end of a block; blocks are seperated by a row of ****
dur_sep = re.compile('[RND][ \d]{2}|[\w.]+')

relations = []
idx = 17 # start index to find the precedence relations
while not re.findall(sep, file[idx]):
    relations.append(file[idx])
    idx += 1

durations = []
idx = idx + 4 # start index to find the task's durations. They start four lines after the relations block.
header = re.findall(dur_sep, file[idx - 2])
while not re.findall(sep, file[idx]):
    durations.append(file[idx].strip().split())
    idx += 1
durations_df = pd.DataFrame(durations, columns=header)

relations_df = pd.DataFrame(columns=relations[0].split())
for i, rel in enumerate(map(str.split, relations[1:])):
    relations_df.loc[i] = [rel[0], rel[1], rel[2], list(map(int, rel[3:]))]

# now merge/join both DataFrames and set the columns types to numeric (when possible)
df = relations_df.merge(durations_df, how='right', on='jobnr.')
df.set_index('jobnr.', inplace=True)

# find all the successors and add them as a new column to each state
succ = {'1':[]}
for index, row in df.iterrows():
    for successor in row['successors']:
        if str(successor) in succ:
            succ[str(successor)].append(int(index))
        else:
            succ[str(successor)] = [int(index)]
df['predecessors'] = df.index.map(succ)

df.index = df.index.map(int)
df = df.apply(pd.to_numeric, errors='ignore')

In [8]:
def get_limits(resources_df):
    limits = pd.concat([resources_df.iloc[3:], resources_df.iloc[3:]], axis = 1)
    limits.columns = ['limits', 'available']
    return limits.apply(pd.to_numeric)

## The input vector